# Paso 0: Dependencias

In [1]:
#configuración en google colab de spark y pyspark
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#!pip install smart-open
#ref: https://radimrehurek.com/gensim/
!pip install gensim
!pip install nltk
!pip install pyLDAvis
# descargar datos 'metadata.csv' de: https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge
# https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge?select=metadata.csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 31.9 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import nltk
import re

In [4]:
# settings en nltk: tokenizador y stopwords
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words_nltk = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
!!unzip gdrive/MyDrive/st1800-231/datasets/metadata50K.csv.zip

['Archive:  gdrive/MyDrive/st1800-231/datasets/metadata50K.csv.zip',
 '  inflating: metadata50K.csv         ']

In [13]:
# cargar datos en pandas
#df = pd.read_csv('metadata.csv')
df = pd.read_csv('gdrive/MyDrive/st1800-231/datasets/metadata50K.csv')

<ipython-input-13-600850fb6bea>:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('gdrive/MyDrive/st1800-231/datasets/metadata50K.csv')


In [ ]:
# analisis descriptivo de datos

In [14]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [15]:
df.shape

(49999, 19)

In [16]:
df.count()

cord_uid            49999
sha                 24700
source_x            49999
title               49967
doi                 49999
pmcid               34636
pubmed_id           29480
license             49999
abstract            33498
publish_time        49999
authors             45185
journal             47835
mag_id                  0
who_covidence_id        0
arxiv_id                0
pdf_json_files      24700
pmc_json_files      20385
url                 49999
s2_id               15363
dtype: int64

In [17]:
# Inicio del proceso LDA
# filtrar las columnas de interés
df = df[['cord_uid','title','abstract']]
df.head()

,cord_uid,title,abstract
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...


In [18]:
# función general para preparación de datos: tokenización, remoción caracteres especiales
# minusculas.... no incluido stemming y lematización.
def textprep(line):
    tokens = nltk.word_tokenize(str(line))
    tokens = [w.lower() for w in tokens if len(w)>1]
    tokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in tokens]
    tokens = [w for w in tokens if w not in stop_words_nltk] 
    return tokens

In [19]:
# creación de columna con tokenización de una columna de interés especifica
df['tokens_title'] = df.apply(lambda row: textprep(row['title']), axis=1)
df.head()

,cord_uid,title,abstract,tokens_title
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,"[clinical, features, cultureproven, mycoplasma..."
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,"[nitric, oxide, proinflammatory, mediator, lun..."
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,"[surfactant, proteind, pulmonary, host, defense]"
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,"[role, endothelin1, lung, disease]"
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,"[gene, expression, epithelial, cells, response..."


### Construir el BoW (diccionario) de términos

In [20]:
# Creación del BoW - en gensim es Dictionary
from gensim.corpora import Dictionary
dictionary = Dictionary(df.tokens_title)

In [21]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in df.tokens_title]

In [22]:
print(corpus[:5])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)], [(19, 1), (20, 1), (21, 1), (22, 1), (23, 1)], [(13, 1), (14, 1), (24, 1), (25, 1)], [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)]]


In [23]:
print(dictionary)

Dictionary(50955 unique tokens: ['abdulaziz', 'arabia', 'clinical', 'cultureproven', 'features']...)


### Construir matriz de documentos vs términos

In [24]:
# libreria para paralelizar
import multiprocessing as mp
import time

t0 = time.time()
pool = mp.Pool(mp.cpu_count())
doc_term_matrix = pool.map(dictionary.doc2bow, [sentence for sentence in df.tokens_title])
pool.close()
print(time.time()-t0)

1.0775258541107178


### Construir modelo LDA

In [25]:
from gensim.models.ldamulticore import LdaMulticore

t0 = time.time()
lda_model = LdaMulticore(doc_term_matrix, num_topics=20, id2word = dictionary, passes=10, workers=10)
print(time.time()-t0)

400.46080565452576


In [26]:
def assigntopic(doc):
    vector = lda_model[dictionary.doc2bow(doc)] 
    # opción 1: todos los tópicos ordenados de mayor a menor, podria ser topN tambien asi: return vector[:5] n=5
    vector = sorted(vector, key=lambda item: -item[1])
    # opción 2: asignar el tópico mayor a cada documento
    #vector = max(vector,key=lambda item: item[1])
    return vector

In [27]:
df['topics'] = df.apply(lambda row: assigntopic(row['tokens_title']), axis=1)
df.head()

,cord_uid,title,abstract,tokens_title,topics
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,"[clinical, features, cultureproven, mycoplasma...","[(5, 0.2594096), (14, 0.23876795), (15, 0.2230..."
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,"[nitric, oxide, proinflammatory, mediator, lun...","[(0, 0.61531675), (14, 0.25611183)]"
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,"[surfactant, proteind, pulmonary, host, defense]","[(8, 0.24731553), (15, 0.22472946), (5, 0.1977..."
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,"[role, endothelin1, lung, disease]","[(3, 0.5543228), (10, 0.26567727), (12, 0.0100..."
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,"[gene, expression, epithelial, cells, response...","[(14, 0.88125)]"


### Ejemplos de tópicos del modelo

In [28]:
# Mostrar los términos y sus pesos de un documento
print(list(lda_model[doc_term_matrix[0]]))

# Mostrar los términos más relevantes de los tópicos más relevantes tópico y sus pesos
print(lda_model.print_topics(num_topics=10, num_words=3))

[(4, 0.2208227), (5, 0.25993022), (14, 0.23870204), (15, 0.22340216)]
[(16, '0.034*"laparoscopic" + 0.021*"surgery" + 0.011*"repair"'), (9, '0.038*"covid19" + 0.030*"pandemic" + 0.016*"health"'), (7, '0.052*"virus" + 0.031*"influenza" + 0.017*"infection"'), (12, '0.015*"abstracts" + 0.012*"news" + 0.011*"society"'), (18, '0.023*"study" + 0.022*"patients" + 0.015*"stroke"'), (19, '0.025*"case" + 0.019*"report" + 0.006*"use"'), (2, '0.018*"covid19" + 0.013*"patients" + 0.011*"trial"'), (3, '0.033*"covid19" + 0.008*"patients" + 0.007*"future"'), (14, '0.021*"virus" + 0.019*"cell" + 0.017*"cells"'), (5, '0.016*"pneumonia" + 0.008*"stent" + 0.008*"using"')]


In [29]:
lda_topic_assignment = [max(p,key=lambda item: item[1]) for p in lda_model[corpus]]

## Visualización de todos los tópicos

In [30]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

t0 = time.time()
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False)
print(time.time()-t0)
vis

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


32.62601375579834


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.078372 -0.068830       1        1  3.625619
1     -0.080220 -0.107190       2        1  4.400478
2     -0.050196 -0.026319       3        1  3.445566
3     -0.122383  0.015319       4        1  4.070323
4     -0.225915  0.056103       5        1  9.919168
5      0.073896  0.007847       6        1  3.045824
6      0.103690  0.004799       7        1  4.601665
7      0.104573  0.141407       8        1  5.676009
8      0.073566  0.027410       9        1  4.139360
9     -0.155399  0.146403      10        1  5.266475
10    -0.066874  0.088752      11        1  3.783301
11    -0.011030  0.062361      12        1  4.748764
12     0.032651 -0.063528      13        1  3.746983
13    -0.034437  0.127177      14        1  4.743734
14     0.248635  0.103011      15        1  9.859617
15    -0.004372 -0.016139      16        1  6.368045
16     0.019492 -0.235382      17        1  5.356041
17     0.004086 -0.035137      18        1  3.229901
18    -0.055597 -0.182067      19        1  6.005896
19     0.067461 -0.045999      20        1  3.967230, topic_info=               Term         Freq        Total Category  logprob  loglift
10783       covid19  5809.000000  5809.000000  Default  30.0000  30.0000
86            virus  2278.000000  2278.000000  Default  29.0000  29.0000
505     respiratory  1575.000000  1575.000000  Default  28.0000  28.0000
104        pandemic  2047.000000  2047.000000  Default  27.0000  27.0000
329          review  1536.000000  1536.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
666    experimental    38.052145   139.524790  Topic20  -6.0542   1.9278
172        analysis    50.701199  1121.145928  Topic20  -5.7672   0.1309
281          versus    43.608137   382.460647  Topic20  -5.9179   1.0557
121        patients    39.561557  2492.979652  Topic20  -6.0153  -0.9163
523           study    37.133742  1722.097566  Topic20  -6.0786  -0.6097

[1487 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
239       2  0.036876      
239       3  0.005673      
239       4  0.080843      
239       5  0.476547      
239       7  0.011346      
...     ...       ...   ...
6824      8  0.758028  zika
6824      9  0.011844  zika
6824     11  0.011844  zika
6824     14  0.035533  zika
6824     15  0.142130  zika

[6346 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

### Guardar la visualización en un archivo HTML

In [31]:
pyLDAvis.save_html(vis, 'lda_visualization.html')

In [32]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaModel

# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=10)

/usr/local/lib/python3.9/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.9/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.9/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [33]:
print(len(common_texts))

9


In [34]:
for t in common_texts:
    print(t)

['human', 'interface', 'computer']
['survey', 'user', 'computer', 'system', 'response', 'time']
['eps', 'user', 'interface', 'system']
['system', 'human', 'system', 'eps']
['user', 'response', 'time']
['trees']
['graph', 'trees']
['graph', 'minors', 'trees']
['graph', 'minors', 'survey']


In [35]:
for text in common_texts:
    print(common_dictionary.doc2bow(text))

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [36]:
print(lda)

LdaModel(num_terms=12, num_topics=10, decay=0.5, chunksize=2000)


In [37]:
# Create a new corpus, made of previously unseen documents.
other_texts = [
    ['computer', 'time', 'graph'],
    ['survey', 'response', 'eps','trees'],
    ['human', 'system', 'computer']
]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]
unseen_doc = other_corpus[1]
vector = lda[unseen_doc]  # get topic probability distribution for a document

In [38]:
print(vector)

[(0, 0.5273133), (1, 0.020000007), (2, 0.020000007), (3, 0.020000007), (4, 0.020000007), (5, 0.02000403), (6, 0.020000007), (7, 0.31267166), (8, 0.020000007), (9, 0.020010997)]


In [39]:
for text in other_texts:
    print(text)

['computer', 'time', 'graph']
['survey', 'response', 'eps', 'trees']
['human', 'system', 'computer']


In [40]:
for text in other_texts:
    print(common_dictionary.doc2bow(text))
for text in other_texts:
    print()

[(0, 1), (6, 1), (10, 1)]
[(3, 1), (4, 1), (8, 1), (9, 1)]
[(0, 1), (1, 1), (5, 1)]





In [41]:
def assigntopic(doc):
    vector = lda[doc] 
    return vector

In [42]:
other_texts = [
    ['computer', 'time', 'graph'],
    ['survey', 'response', 'eps','trees'],
    ['human', 'system', 'computer']
]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]
unseen_doc = other_corpus[1]
vector = lda[unseen_doc]  # get topic probability distribution for a document